[![LibKet](../images/LibKet.png)](https://gitlab.com/mmoelle1/LibKet)
**LibKet - The Quantum Expression Template Library.**
- Repository:    https://gitlab.com/mmoelle1/LibKet/
- Documentation: https://libket.readthedocs.io/
- API docs:      https://mmoelle1.gitlab.io/LibKet/

***

# Tutorial \#3: Hands-on Scientific Computing with LibKet - Part 2

> In this tutorial you will learn to
> 1. write quantum expressions with for loops
> 2. define custom quantum gates
> 3. build quantum algorithms from component

## Getting started
Let's include **LibKet**'s main headerfile, import its namespaces, and inject the code for displaying images. This can take some time, stay tuned.

In [2]:
#include "LibKet.hpp"
using namespace LibKet;
using namespace LibKet::circuits;
using namespace LibKet::filters;
using namespace LibKet::gates;

A common building block in many quantum algorithms is the [Quantum Fourier Transform](https://en.wikipedia.org/wiki/Quantum_Fourier_transform) (QFT) and its inverse QFT$^\dagger$

![QFT](../images/qft_circuit.png)

**LibKet** has ready-to-use quantum expressions ``qft(...)`` and ``qftdag(...)``. Let's start with the former one.

In [9]:
auto expr = LibKet::circuits::qft<LibKet::circuits::QFTMode::noswap>(init());

QDevice<QDeviceType::qiskit_qasm_simulator, 4> qiskit;
std::cout << qiskit(expr) << std::endl;

OPENQASM 2.0;
include "qelib1.inc";
qreg q[4];
creg c[4];
h q[0];
cu1(1.570796326794896558) q[1], q[0];
cu1(0.785398163397448279) q[2], q[0];
cu1(0.392699081698724139) q[3], q[0];
h q[1];
cu1(1.570796326794896558) q[2], q[1];
cu1(0.785398163397448279) q[3], q[1];
h q[2];
cu1(1.570796326794896558) q[3], q[2];
h q[3];



The pre-implemented ``qft(...)`` function make use of another **LibKet** feature

## Compile-time for loops with ``static_for()``

The generic interface of the ``static_for()`` function reads
```cpp
template<index_t for_start,
         index_t for_end,
         index_t for_step,
         template<index_t start, index_t end, index_t step, index_t index>
         class functor,
         typename functor_return_type,
         typename... functor_types>
inline auto
static_for(functor_return_type&& functor_return_arg,
           functor_types&&... functor_args)
```
which might look complicated at first glance.

Let's have a look at a minimalistic example. We first create the functor

In [ ]:
template<index_t start, index_t end, index_t step, index_t index>
struct ftor 
{
    template<typename  Expr>
    inline constexpr auto operator()(Expr&& expr) noexcept 
    {
        // Returns the controlled phase shift gate with angle
        // theta = pi/2^k between qubits index-1 and index
        return crk<index>(sel<index-1>(gototag<0>()),
                          sel<index  >(gototag<0>(expr)));
    }
};

Let us *loop* through this functor for ``index = 1,...,4``

In [ ]:
auto expr = utils::static_for<1,5,1,ftor>(tag<0>(init()));

QDevice<QDeviceType::qiskit_qasm_simulator, 4> qiskit;
std::cout << qiskit(expr) << std::endl;

Compile-time for loops can be nested as it is done in our QFT implementation

In [ ]:
template<index_t start, index_t end, index_t step, index_t index>
struct qft_loop_inner
{
    template<typename Expr0, typename Expr1>
    inline constexpr auto operator()(Expr0&& expr0, Expr1&& expr1) noexcept
    {
        return crk<index-start+2>(sel<index  >(gototag<0>(expr0)),
                                  sel<start-1>(gototag<0>(expr1)));
    }
};

template<index_t start, index_t end, index_t step, index_t index>
struct qft_loop_outer
{
    template<typename Expr0, typename Expr1>
    inline constexpr auto operator()(Expr0&& expr0, Expr1&& expr1) noexcept
    {
        return utils::static_for<index+1, end, 1, qft_loop_inner>
            (h(sel<index>(gototag<0>(expr0))), gototag<0>(expr1));
    }
};